In [14]:
import pandas as pd
from tensorflow.keras.datasets import reuters
from tensorflow.keras.utils import to_categorical
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import nltk
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Ensure NLTK resources are downloaded
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Load the Reuters dataset
from tensorflow.keras.datasets import reuters
from tensorflow.keras.utils import to_categorical

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(
    path="reuters.npz",
    num_words=None,
    skip_top=0,
    maxlen=None,
    test_split=0.2,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3
)

# Convert labels to categorical format
num_classes = max(train_labels) + 1
train_labels = to_categorical(train_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

# Convert one-hot encoded labels back to integer labels
train_labels_int = train_labels.argmax(axis=1)
test_labels_int = test_labels.argmax(axis=1)

# Get the word index and invert it to map indices to words
word_index = reuters.get_word_index()
index_to_word = {index: word for word, index in word_index.items()}

# Decoding function
def decode_review(encoded_review):
    return ' '.join([index_to_word.get(i - 3, '?') for i in encoded_review])

# Decode the train and test data
train_texts = [decode_review(sequence) for sequence in train_data]
test_texts = [decode_review(sequence) for sequence in test_data]

# NLP Preprocessing
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Tokenization
    tokens = word_tokenize(text)
    # Removing stop words and punctuation
    tokens = [word for word in tokens if word.isalnum() and word not in stopwords.words('english')]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

# Apply preprocessing
train_texts = [preprocess_text(text) for text in train_texts]
test_texts = [preprocess_text(text) for text in test_texts]

# Load Word2Vec model and vectors
word2vec_model = KeyedVectors.load("C:\SNLP\word2vec-google-news-300.kv")  # Adjust path
word2vec_vectors = np.load("C:\SNLP\word2vec-google-news-300.kv.vectors.npy")  # Adjust path

# Function to convert text to Word2Vec vectors
def text_to_word2vec(text, model):
    tokens = word_tokenize(text)
    vectors = [model[word] for word in tokens if word in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)  # Return zero vector if no words are in the model

# Convert train and test texts to Word2Vec vectors
X_train_w2v = np.array([text_to_word2vec(text, word2vec_model) for text in train_texts])
X_test_w2v = np.array([text_to_word2vec(text, word2vec_model) for text in test_texts])

# Standardize the feature vectors
scaler = StandardScaler()
X_train_w2v = scaler.fit_transform(X_train_w2v)
X_test_w2v = scaler.transform(X_test_w2v)

# Train a Logistic Regression model with softmax
log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs')
log_reg.fit(X_train_w2v, train_labels_int)

# Predict and evaluate
y_pred = log_reg.predict(X_test_w2v)
print(classification_report(test_labels_int, y_pred, target_names=[f'Class {i}' for i in range(num_classes)]))

# Display example predictions
print("Example predictions:")
for i in range(5):
    print(f"Text: {test_texts[i]}")
    print(f"Predicted Label: {y_pred[i]}")
    print(f"True Label: {test_labels_int[i]}")
    print()


In [2]:
from tensorflow.keras.datasets import reuters
from tensorflow.keras.utils import to_categorical

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(
    path="reuters.npz",
    num_words=None,
    skip_top=0,
    maxlen=None,
    test_split=0.2,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3
)

In [3]:
# Convert labels to categorical format
num_classes = max(train_labels) + 1
train_labels = to_categorical(train_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

# Convert one-hot encoded labels back to integer labels
train_labels_int = train_labels.argmax(axis=1)
test_labels_int = test_labels.argmax(axis=1)

# Get the word index and invert it to map indices to words
word_index = reuters.get_word_index()
index_to_word = {index: word for word, index in word_index.items()}


In [4]:
def decode_review(encoded_review):
    return ' '.join([index_to_word.get(i - 3, '?') for i in encoded_review])

# Decode the train and test data
train_texts = [decode_review(sequence) for sequence in train_data]
test_texts = [decode_review(sequence) for sequence in test_data]


In [5]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stop words
    tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(tokens)

# Apply preprocessing
train_texts = [preprocess_text(text) for text in train_texts]
test_texts = [preprocess_text(text) for text in test_texts]

In [7]:
word2vec_model = KeyedVectors.load("C:\SNLP\word2vec-google-news-300.kv")  # Adjust path
word2vec_vectors = np.load("C:\SNLP\word2vec-google-news-300.kv.vectors.npy")  # Adjust path

# Function to convert text to Word2Vec vectors
def text_to_word2vec(text, model):
    tokens = word_tokenize(text)
    vectors = [model[word] for word in tokens if word in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

In [8]:
X_train_w2v = np.array([text_to_word2vec(text, word2vec_model) for text in train_texts])
X_test_w2v = np.array([text_to_word2vec(text, word2vec_model) for text in test_texts])

# Standardize the feature vectors
scaler = StandardScaler()
X_train_w2v = scaler.fit_transform(X_train_w2v)
X_test_w2v = scaler.transform(X_test_w2v)

# Train a Logistic Regression model with softmax
log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs')
log_reg.fit(X_train_w2v, train_labels_int)

D:\Anaconda\envs\trv2\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

In [9]:
y_pred = log_reg.predict(X_test_w2v)
print(classification_report(test_labels_int, y_pred, target_names=[f'Class {i}' for i in range(num_classes)]))

# Display example predictions
print("Example predictions:")
for i in range(5):
    print(f"Text: {test_texts[i]}")
    print(f"Predicted Label: {y_pred[i]}")
    print(f"True Label: {test_labels_int[i]}")
    print()

              precision    recall  f1-score   support

     Class 0       0.88      0.58      0.70        12
     Class 1       0.66      0.71      0.69       105
     Class 2       0.83      0.75      0.79        20
     Class 3       0.91      0.92      0.92       813
     Class 4       0.84      0.84      0.84       474
     Class 5       0.50      0.20      0.29         5
     Class 6       0.86      0.86      0.86        14
     Class 7       1.00      0.33      0.50         3
     Class 8       0.56      0.66      0.60        38
     Class 9       0.77      0.80      0.78        25
    Class 10       0.83      0.83      0.83        30
    Class 11       0.64      0.69      0.66        83
    Class 12       0.62      0.62      0.62        13
    Class 13       0.62      0.57      0.59        37
    Class 14       0.33      0.50      0.40         2
    Class 15       0.83      0.56      0.67         9
    Class 16       0.63      0.74      0.68        99
    Class 17       0.82    

In [12]:
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',  # Softmax for multi-class classification
    num_class=num_classes,      # Number of classes
    n_estimators=100,           # Number of boosting rounds
    use_label_encoder=False     # Avoid using deprecated label encoder
)
xgb_model.fit(X_train_w2v, train_labels_int)

# Predict and evaluate
y_pred = xgb_model.predict(X_test_w2v)
print(classification_report(test_labels_int, y_pred, target_names=[f'Class {i}' for i in range(num_classes)]))

# Display example predictions
print("Example predictions:")
for i in range(5):
    print(f"Text: {test_texts[i]}")
    print(f"Predicted Label: {y_pred[i]}")
    print(f"True Label: {test_labels_int[i]}")
    print()

              precision    recall  f1-score   support

     Class 0       0.67      0.50      0.57        12
     Class 1       0.64      0.70      0.67       105
     Class 2       0.72      0.65      0.68        20
     Class 3       0.89      0.91      0.90       813
     Class 4       0.78      0.87      0.82       474
     Class 5       0.00      0.00      0.00         5
     Class 6       0.80      0.86      0.83        14
     Class 7       1.00      0.33      0.50         3
     Class 8       0.63      0.68      0.66        38
     Class 9       0.68      0.68      0.68        25
    Class 10       0.72      0.60      0.65        30
    Class 11       0.64      0.77      0.70        83
    Class 12       0.50      0.23      0.32        13
    Class 13       0.53      0.54      0.53        37
    Class 14       1.00      0.50      0.67         2
    Class 15       0.67      0.22      0.33         9
    Class 16       0.63      0.69      0.66        99
    Class 17       0.50    

D:\Anaconda\envs\trv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Anaconda\envs\trv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Anaconda\envs\trv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train_w2v.shape[1],)),
    Dense(256, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_w2v, train_labels, epochs=10, batch_size=32, validation_split=0.2)

# Predict and evaluate
y_pred = model.predict(X_test_w2v)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(test_labels_int, y_pred_classes, target_names=[f'Class {i}' for i in range(num_classes)]))

# Display example predictions
print("Example predictions:")
for i in range(5):
    print(f"Text: {test_texts[i]}")
    print(f"Predicted Label: {y_pred_classes[i]}")
    print(f"True Label: {test_labels_int[i]}")
    print()

Epoch 1/10
225/225 [==============================] - 3s 9ms/step - loss: 1.1145 - accuracy: 0.7478 - val_loss: 0.8157 - val_accuracy: 0.8024
Epoch 2/10
225/225 [==============================] - 1s 5ms/step - loss: 0.6866 - accuracy: 0.8248 - val_loss: 0.7809 - val_accuracy: 0.8208
Epoch 3/10
225/225 [==============================] - 1s 4ms/step - loss: 0.5452 - accuracy: 0.8624 - val_loss: 0.7705 - val_accuracy: 0.8152
Epoch 4/10
225/225 [==============================] - 1s 5ms/step - loss: 0.4537 - accuracy: 0.8799 - val_loss: 0.8058 - val_accuracy: 0.8158
Epoch 5/10
225/225 [==============================] - 1s 4ms/step - loss: 0.3840 - accuracy: 0.8966 - val_loss: 0.8528 - val_accuracy: 0.8052
Epoch 6/10
225/225 [==============================] - 1s 4ms/step - loss: 0.3208 - accuracy: 0.9088 - val_loss: 0.8977 - val_accuracy: 0.8052
Epoch 7/10
225/225 [==============================] - 1s 5ms/step - loss: 0.2923 - accuracy: 0.9187 - val_loss: 0.8771 - val_accuracy: 0.8097
Epoch 

D:\Anaconda\envs\trv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Anaconda\envs\trv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Anaconda\envs\trv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
